In [14]:
import pandas as pd
import numpy as np
import os


def P(df, col):
    """
    Calculate the probability of each unique value in a column of a DataFrame and return a mapping the value to its probability.
    """
    # Calculate the probability of each unique value in the column
    prob = df[col].value_counts(normalize=True).to_dict()
    # Create a mapping of the value to its probability
    mapping = {k: v for k, v in prob.items()}
    return mapping

df = pd.read_csv(f'{os.getcwd()}/cleaned_data.csv')
df

C:\Users\franj\AppData\Local\Temp\ipykernel_31148\539358981.py:16: DtypeWarning: Columns (1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{os.getcwd()}/cleaned_data.csv')


,ship_company,train_operator,container_type,dangerous_goods,custom,waste,full_empty,content,weight,train_id,...,month_of_arrival,container_owner_prefix,equipment_identifier,weather_today,weather_prev_day,weather_next_day_1,weather_next_day_2,weather_next_day_3,weather_next_day_4,weather_next_day_5
0,MAE-HAM,BOX,20G,0,0,0,1,NaN,2826.0,BRV13837Z,...,January,HAS,U,Regen,Niesel,Unwetter,Niesel,Schnee,Schnee,Schnee
1,MAE-HAM,BOX,40GH,0,0,0,1,NaN,7331.0,BRV13837Z,...,January,MSK,U,Regen,Niesel,Unwetter,Niesel,Schnee,Schnee,Schnee
2,MAE-HAM,BOX,40GH,0,0,0,1,NaN,8815.0,BRV13837Z,...,January,GES,U,Regen,Niesel,Unwetter,Niesel,Schnee,Schnee,Schnee
3,MAE-HAM,BOX,20G,0,0,0,1,NaN,15492.0,BRV13837Z,...,January,HAS,U,Regen,Niesel,Unwetter,Niesel,Schnee,Schnee,Schnee
4,MAE-HAM,BOX,40GH,0,0,0,1,NaN,5778.0,BRV13837Z,...,January,MRK,U,Regen,Niesel,Unwetter,Niesel,Schnee,Schnee,Schnee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276146,TFG,NaN,40GH,0,0,0,1,NaN,5510.0,NaN,...,December,ONE,U,Regen,Regen,Unwetter,Regen,Regen,Regen,Regen
276147,TFG,NaN,40GH,0,0,0,1,NaN,8286.0,NaN,...,December,SEL,U,Regen,Regen,Unwetter,Regen,Regen,Regen,Regen
276148,TFG,NaN,40GH,0,0,0,1,NaN,3802.0,NaN,...,December,FFA,U,Regen,Regen,Unwetter,Regen,Regen,Regen,Regen
276149,TFG,NaN,40GH,0,0,0,1,NaN,5662.0,NaN,...,December,HAM,U,Regen,Regen,Unwetter,Regen,Regen,Regen,Regen


# Container Parameter Distributions

In [25]:
unittypes = pd.read_csv(f'{os.getcwd()}/unittype.csv')

# calculate probability of container type ending with 'HC'
df['container_type_HC'] = df['container_type'].str.endswith('40HC').astype(int)
print(P(df, 'container_type_HC'))

# Create a new column 'unit_type' in df and do a join of container type onto unittype and return category
df['unit_type'] = df['container_type'].map(unittypes.set_index('Unittype')['Category'])
df['unit_type'] = df['unit_type'].fillna('Unknown')
df['unit_type'] = df['unit_type'].replace('', 'Unknown')

# Drop rows with 'Unknown' in 'unit_type'
df = df[df['unit_type'] != 'Unknown']
df = df[df['unit_type'] != '''45' Container''']

df['unit_type'].value_counts()
print(P(df, 'unit_type'))

# For each unique value in 'unit_type', calculate the maximum weight and the average weight of containers
df['max_weight'] = df.groupby('unit_type')['weight'].transform('max')
df['avg_weight'] = df.groupby('unit_type')['weight'].transform('mean')

print(df[['unit_type', 'max_weight', 'avg_weight']].drop_duplicates().sort_values(by='unit_type'))



{0: 0.8885147746653101, 1: 0.11148522533468991}
{"40' Container": 0.5219808372188348, 'Wechselbrücke': 0.2512022485891421, "20' Container": 0.17697757998521435, 'Trailer': 0.03182207452843309, "30' Container": 0.018017259678375627}
         unit_type  max_weight    avg_weight
0    20' Container     31000.0  12027.625969
506  30' Container     31000.0  22629.840951
1    40' Container     31000.0   9658.038801
513        Trailer     31000.0  16047.452099
270  Wechselbrücke     31000.0   7222.645197


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KernelDensity
import pickle
from datetime import datetime
import os

# Create directory for saving models and plots
os.makedirs('models', exist_ok=True)
os.makedirs('plots', exist_ok=True)

# Function to convert datetime to hours (0-24)
def datetime_to_decimal_hours(dt):
    if pd.isna(dt):
        return np.nan
    if isinstance(dt, str):
        dt = pd.to_datetime(dt)
    return dt.hour + dt.minute/60 + dt.second/3600

# Function to calculate arrival delay in hours
def calculate_delay(planned, realized):
    if pd.isna(planned) or pd.isna(realized):
        return np.nan
    if isinstance(planned, str):
        planned = pd.to_datetime(planned)
    if isinstance(realized, str):
        realized = pd.to_datetime(realized)
    delay = (realized - planned).total_seconds() / 3600  # delay in hours
    return delay

# Convert datetime columns to pandas datetime
datetime_columns = ['planned_arrival_train', 'realized_arrival_train', 'pickup_realized']
for col in datetime_columns:
    df[col] = pd.to_datetime(df[col])

# Extract time of day (0-24 hours) for each event
df['planned_hour'] = df['planned_arrival_train'].apply(datetime_to_decimal_hours)
df['realized_hour'] = df['realized_arrival_train'].apply(datetime_to_decimal_hours)
df['pickup_hour'] = df['pickup_realized'].apply(datetime_to_decimal_hours)

# Calculate delays
df['train_delay_hours'] = df.apply(lambda row: calculate_delay(row['planned_arrival_train'], 
                                                             row['realized_arrival_train']), axis=1)

# Calculate time from train arrival to truck pickup (in hours)
df['wait_for_pickup_hours'] = df.apply(lambda row: calculate_delay(row['realized_arrival_train'], 
                                                                 row['pickup_realized']), axis=1)

# Improved function to create and save KDE model with better visualization
def create_and_save_kde(data, name, bandwidth=0.5, x_min=0, x_max=24, bins=48):
    """
    Create and save a KDE model with improved visualization
    
    Args:
        data: The array of values to model
        name: Name prefix for saved files
        bandwidth: Bandwidth parameter for KDE
        x_min, x_max: Range for x-axis
        bins: Number of bins for histogram
    """
    # Remove any NaN values
    clean_data = data.dropna()
    
    # Reshape data for KDE
    data_reshaped = clean_data.values.reshape(-1, 1)
    
    # Create KDE model
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth)
    kde.fit(data_reshaped)
    
    # Save the model
    with open(f'models/{name}_kde.pkl', 'wb') as f:
        pickle.dump(kde, f)
    
    # Create visualization
    plt.figure(figsize=(12, 6))
    
    # Calculate histogram values to use for KDE scaling
    hist, bin_edges = np.histogram(clean_data, bins=bins, range=(x_min, x_max))
    max_hist_val = np.max(hist)
    
    # Plot histogram
    n, bins, patches = plt.hist(clean_data, bins=bins, range=(x_min, x_max), 
                               alpha=0.7, color='skyblue', edgecolor='black', linewidth=0.5)
    
    # Calculate KDE on a fine grid
    x_grid = np.linspace(x_min, x_max, 1000).reshape(-1, 1)
    log_dens = kde.score_samples(x_grid)
    dens = np.exp(log_dens)
    
    # Scale KDE to match histogram height
    dens_scaled = dens * (max_hist_val / np.max(dens))
    
    # Plot the scaled KDE
    plt.plot(x_grid, dens_scaled, 'r-', lw=2, label=f'{name} KDE')
    
    # Set nice hour labels for time-based plots
    if x_max == 24 and x_min == 0:
        hour_labels = [f"{h:02d}:00" for h in range(0, 25, 3)]
        plt.xticks(np.arange(0, 25, 3), hour_labels)
    
    plt.title(f'Distribution of {name}')
    plt.xlabel('Time of Day' if x_max == 24 else 'Hours')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Save the plot
    plt.savefig(f'plots/{name}_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return kde

# Create improved KDEs with appropriate bandwidths
train_arrival_kde = create_and_save_kde(df['realized_hour'], 'train_arrival', bandwidth=0.3)
truck_pickup_kde = create_and_save_kde(df['pickup_hour'], 'truck_pickup', bandwidth=0.3)

# Handle the train delay differently since it has that spike at 0
# For train delay, we'll handle it differently since it's not a time of day
train_delay_data = df['train_delay_hours'].dropna()
train_delay_range = (-3, 3)  # Looking at delays from -3 to +3 hours

# Plot train delay with appropriate scaling
plt.figure(figsize=(12, 6))
n, bins, patches = plt.hist(train_delay_data.clip(*train_delay_range), bins=50, 
                           alpha=0.7, color='skyblue', edgecolor='black', linewidth=0.5)

# Create KDE for train delay (smaller bandwidth for better detail)
delay_kde = KernelDensity(kernel='gaussian', bandwidth=0.1)
delay_kde.fit(train_delay_data.clip(*train_delay_range).values.reshape(-1, 1))

# Save the model
with open(f'models/train_delay_kde.pkl', 'wb') as f:
    pickle.dump(delay_kde, f)

# Plot KDE
x_grid = np.linspace(*train_delay_range, 1000).reshape(-1, 1)
log_dens = delay_kde.score_samples(x_grid)
dens = np.exp(log_dens)
dens_scaled = dens * (np.max(n) / np.max(dens))
plt.plot(x_grid, dens_scaled, 'r-', lw=2, label='Train Delay KDE')

plt.title('Distribution of Train Delays (hours)')
plt.xlabel('Delay (hours)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('plots/train_delay_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

# Create and save KDE for pickup wait times (use only reasonable wait times)
wait_data = df['wait_for_pickup_hours'].dropna()
wait_range = (0, 48)  # 0 to 48 hours wait time
pickup_wait_kde = create_and_save_kde(wait_data.clip(*wait_range), 'pickup_wait', 
                                     bandwidth=1.0, x_min=wait_range[0], x_max=wait_range[1], bins=50)

# Create a combined visualization for train and truck time distributions
plt.figure(figsize=(14, 7))

# Create histograms
plt.hist(df['realized_hour'].dropna(), bins=48, alpha=0.5, color='blue', label='Train Arrivals')
plt.hist(df['pickup_hour'].dropna(), bins=48, alpha=0.5, color='green', label='Truck Pickups')

# Calculate and plot KDEs
x_grid = np.linspace(0, 24, 1000).reshape(-1, 1)

# Get the maximum height of histograms for scaling
train_hist, _ = np.histogram(df['realized_hour'].dropna(), bins=48, range=(0, 24))
truck_hist, _ = np.histogram(df['pickup_hour'].dropna(), bins=48, range=(0, 24))
max_hist_val = max(np.max(train_hist), np.max(truck_hist))

# Train KDE
log_dens_train = train_arrival_kde.score_samples(x_grid)
dens_train = np.exp(log_dens_train)
dens_train_scaled = dens_train * (max_hist_val / np.max(dens_train))
plt.plot(x_grid, dens_train_scaled, 'b-', lw=2, label='Train Arrivals KDE')

# Truck KDE
log_dens_truck = truck_pickup_kde.score_samples(x_grid)
dens_truck = np.exp(log_dens_truck)
dens_truck_scaled = dens_truck * (max_hist_val / np.max(dens_truck))
plt.plot(x_grid, dens_truck_scaled, 'g-', lw=2, label='Truck Pickups KDE')

# Add 24-hour clock labels
hour_labels = [f"{h:02d}:00" for h in range(0, 25, 3)]
plt.xticks(np.arange(0, 25, 3), hour_labels)

plt.title('Distribution of Train Arrivals and Truck Pickups (24-hour)')
plt.xlabel('Time of Day')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('plots/combined_distributions.png', dpi=300, bbox_inches='tight')
plt.close()

print("KDE models and visualizations have been created and saved.")

# Create a function to sample from the KDE models with normalization
def sample_from_kde(kde_model, n_samples=1, min_val=0, max_val=24):
    """
    Sample values from a KDE model with bounds
    
    Args:
        kde_model: The trained KDE model
        n_samples: Number of samples to generate
        min_val: Minimum allowed value
        max_val: Maximum allowed value
    
    Returns:
        Array of sampled values
    """
    samples = kde_model.sample(n_samples=n_samples)
    
    # Apply modulo 24 to ensure time values stay within 0-24 range
    if min_val == 0 and max_val == 24:
        samples = samples % 24
    
    # Clip values to specified range
    samples = np.clip(samples, min_val, max_val)
    
    return samples.flatten()

# Example usage
print("\nExample: Generating sample train arrival times:")
train_samples = sample_from_kde(train_arrival_kde, n_samples=5)
print(f"Sample train arrival times (hours): {train_samples}")

print("\nExample: Generating sample truck pickup times:")
truck_samples = sample_from_kde(truck_pickup_kde, n_samples=5)
print(f"Sample truck pickup times (hours): {truck_samples}")

# Save helper functions for the environment
with open('models/kde_sampling_utils.py', 'w') as f:
    f.write("""
import numpy as np
import pickle

def load_kde_model(model_path):
    with open(model_path, 'rb') as f:
        return pickle.load(f)

def sample_from_kde(kde_model, n_samples=1, min_val=0, max_val=24):
    samples = kde_model.sample(n_samples=n_samples)
    
    # Apply modulo 24 to ensure time values stay within 0-24 range
    if min_val == 0 and max_val == 24:
        samples = samples % 24
    
    # Clip values to specified range
    samples = np.clip(samples, min_val, max_val)
    
    return samples.flatten()

def hours_to_time(hours):
    h = int(hours)
    m = int((hours - h) * 60)
    s = int(((hours - h) * 60 - m) * 60)
    return h, m, s
""")

KDE models and visualizations have been created and saved.

Example: Generating sample train arrival times:
Sample train arrival times (hours): [15.73414199 17.98819054  1.10468822 19.50063177  4.21768477]

Example: Generating sample truck pickup times:
Sample truck pickup times (hours): [11.12534527 10.54106944 15.00921458  3.75283153  9.86949035]


In [21]:
# Add to the KDE creation script

def create_weight_kde(data, bandwidth=500):
    """
    Create a KDE model for container weights.
    
    Args:
        data: DataFrame containing weight column
        bandwidth: Bandwidth for the KDE model
    
    Returns:
        Trained KDE model
    """
    # Filter out any negative weights or extremely large outliers
    filtered_weights = data['weight'].dropna()
    filtered_weights = filtered_weights[(filtered_weights > 0) & (filtered_weights < 40000)]
    
    # Reshape for KDE
    weights_reshaped = filtered_weights.values.reshape(-1, 1)
    
    # Create KDE model
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth)
    kde.fit(weights_reshaped)
    
    # Save the model
    with open('models/container_weight_kde.pkl', 'wb') as f:
        pickle.dump(kde, f)
    
    # Create visualization
    plt.figure(figsize=(12, 6))
    
    # Plot histogram
    n, bins, patches = plt.hist(filtered_weights, bins=50, alpha=0.7, 
                              color='skyblue', edgecolor='black', linewidth=0.5)
    
    # Plot KDE
    x_grid = np.linspace(0, 40000, 1000).reshape(-1, 1)
    log_dens = kde.score_samples(x_grid)
    dens = np.exp(log_dens)
    
    # Scale KDE to match histogram height
    dens_scaled = dens * (np.max(n) / np.max(dens))
    
    plt.plot(x_grid, dens_scaled, 'r-', lw=2, label='Container Weight KDE')
    
    plt.title('Distribution of Container Weights')
    plt.xlabel('Weight (kg)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Save the plot
    plt.savefig('plots/container_weight_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return kde

# Add this to your main KDE creation script
print("Creating container weight KDE model...")
weight_kde = create_weight_kde(df)
print("Container weight KDE model created and saved.")

Creating container weight KDE model...
Container weight KDE model created and saved.


In [ ]:
# First, let's calculate P(unit_type | dangerous_goods)
# This shows the distribution of container types given dangerous goods status
p_unit_type_given_dg = {}
for dg in [0, 1]:  # 0 = non-dangerous, 1 = dangerous
    # Filter dataframe by dangerous_goods value
    filtered_df = df[df['dangerous_goods'] == dg]
    # Calculate probability distribution within this filtered dataset
    p_unit_type_given_dg[dg] = P(filtered_df, 'unit_type')
    
print(f"P(unit_type | dangerous_goods=0):")
print(p_unit_type_given_dg[0])
print(f"\nP(unit_type | dangerous_goods=1):")
print(p_unit_type_given_dg[1])

# Now calculate P(dangerous_goods | unit_type)
# This shows the probability of dangerous goods for each container type
p_dg_given_unit_type = {}
for unit_type in df['unit_type'].unique():
    # Filter dataframe by unit_type
    filtered_df = df[df['unit_type'] == unit_type]
    # Calculate probability distribution within this filtered dataset
    p_dg_given_unit_type[unit_type] = P(filtered_df, 'dangerous_goods')

print("\nP(dangerous_goods | unit_type):")
for unit_type, probs in p_dg_given_unit_type.items():
    print(f"{unit_type}: {probs}")
# remove all entries where the container type does not start with '20' or '40'
df = df[df['unit_type'].str.startswith(('20', '40', 'Wechselbrücke', 'Trailer')).copy()]
print('Container type distribution')
print(P(df, 'unit_type'))

# return probs for dangerous goods
print('Dangerous goods distribution')
p_dg = P(df, 'dangerous_goods')
print(p_dg)

# dangerous goods have 15 places in the yard, reefer containers have 10 places in the yard
# reefer containers have 2/3 of the places in the yard
print('Reefer container distribution')
p_ref = {1-p_dg[1]*2/3, p_dg[1]*2/3}
print(p_ref)

#TODO: create distribution of container heights
#TODO: create distribution of container weights

P(unit_type | dangerous_goods=0):
{"40' Container": 0.5223409956903831, 'Wechselbrücke': 0.2481370267313721, "20' Container": 0.17513784166746202, 'Trailer': 0.0296020028339219, "30' Container": 0.01408885005910122, "45' Container": 0.010689612134471796, 'Unknown': 3.6708832879367434e-06}

P(unit_type | dangerous_goods=1):
{"30' Container": 0.29033984479529035, 'Wechselbrücke': 0.2788332887342788, "20' Container": 0.17313352956917313, 'Trailer': 0.16885202033716884, "40' Container": 0.08884131656408883}

P(dangerous_goods | unit_type):
20' Container: {0: 0.9866203445209587, 1: 0.013379655479041297}
40' Container: {0: 0.997672217353199, 1: 0.0023277826468010515}
Wechselbrücke: {0: 0.9848189049797488, 1: 0.015181095020251173}
45' Container: {0: 1.0}
30' Container: {0: 0.7796059313426772, 1: 0.22039406865732278}
Trailer: {0: 0.9274295572167912, 1: 0.07257044278320875}
Unknown: {0: 1.0}
Container type distribution
{"40' Container": 0.5315580567616421, 'Wechselbrücke': 0.2558112666082776, "

In [ ]:
#TODO: Distribution of truck arrival times
#TODO: Distribution of train arrival times
#TODO: Distribution of delay times
#TODO: Duration of stay for trains in yard
#TODO: Average duration of stay for trucks in yard